In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

In [3]:
%matplotlib inline

In [4]:
TEXT_COLOUR = {
    'PURPLE':'\033[95m',
    'CYAN':'\033[96m',
    'DARKCYAN':'\033[36m',
    'BLUE':'\033[94m',
    'GREEN':'\033[92m',
    'YELLOW':'\033[93m',
    'RED':'\033[91m',
    'BOLD':'\033[1m',
    'UNDERLINE':'\033[4m',
    'END':'\033[0m'
}

def print_bold(*msgs):
    print(TEXT_COLOUR['BOLD'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_green(*msgs):
    print(TEXT_COLOUR['GREEN'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_error(*msgs):
    print(TEXT_COLOUR['RED'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def wrap_green(msg):
    return TEXT_COLOUR['GREEN'] + msg + TEXT_COLOUR['END']

def wrap_red(msg):
    return TEXT_COLOUR['RED'] + msg + TEXT_COLOUR['END']

def up_down_str(val):
    msg = str(val)
    if val > 0:
        msg = wrap_green(msg)
    elif val < 0:
        msg = wrap_red(msg)
    return msg

In [5]:
exp='roberta-large'
num_layers = 24

In [6]:
# tasks = ["CoLA","SST-2","MRPC","STS-B","QQP","MNLI", "MNLI-MM", "QNLI","RTE"]
tasks = ["CoLA","SST-2","MRPC","STS-B","RTE"]

metrics = {
    "CoLA":["mcc"],
    "MNLI":["acc"],
    "MNLI-MM":["acc"],
    "MRPC":["f1"],
    "QNLI":["acc"],
    "QQP":["f1"],
    "RTE":["acc"],
    "SST-2":["acc"],
    "STS-B":["spearmanr"],
    "WNLI":["acc"] #temp
}

reported_in_paper = {
    "CoLA":0.00,
    "MNLI":0.00,
    "MNLI-MM":0.0,
    "MRPC":0.00,
    "QNLI":0.00,
    "QQP":0.00,
    "RTE":0.00,
    "SST-2":0.00,
    "STS-B":0.00,
    "WNLI":0.00
}

In [7]:

def get_average_val(lines):
    reported = []
    for line in lines:
#         print('\t', line)
        val = float(line.split()[1])
        if val != 0:
            reported.append(val)
    out = 0
    if len(reported) != 0:
        reported.sort(reverse = True)
        candidates = [reported[0]]
        for j in range(1, len(reported)):
            if reported[j] > 0.9 * reported[0]:
                candidates.append(reported[j])
        out = np.mean(candidates)
        
    return out


In [8]:
results = {}

for task in tasks:
    task_results = {}
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        # base metrics
        f=open(f"../../mt_dnn_exp_results/{exp}/{task}/base-{metric}.txt", "r")
        lines = f.read().splitlines()
        task_results[f'base-{metric}'] = get_average_val(lines)
        
        # no layer metrics
        
        fine_tuning_metrics = []
        f=open(f"../../mt_dnn_exp_results/{exp}/{task}/no_layer-{metric}.txt", "r")

        lines = f.read().splitlines()
        fine_tuning_metrics.append(get_average_val(lines))
        
        # fine-tuned metrics
        
        log_file_prefix=''
        for i in reversed(range(int(num_layers/2), num_layers)):
            log_file_prefix += str(i)
            f=open(f"../../mt_dnn_exp_results/{exp}/{task}/{log_file_prefix}-{metric}.txt", "r")
            lines = f.read().splitlines()
            fine_tuning_metrics.append(get_average_val(lines))
            
            log_file_prefix +='_'
        
        task_results[f'{metric}'] = list(reversed(fine_tuning_metrics))
        
    results[task] = task_results

In [9]:
x_axis = []

for i in range(int(num_layers/2), num_layers):
    x_axis.append(str(i))

x_axis.append("none")

In [10]:
def draw_graph(task, y_label, paper, base, reported):
    plt.figure(figsize=(10,6))
    plt.plot(x_axis, reported)
    
    plt.xlabel("layers")
    plt.ylabel(y_label)
    
    if paper == 0.0:    
        gap = max(reported) - min(reported)
        top = max(max(reported), base) + (gap*0.2)
        bottom = min(min(reported), base) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
    else:
        gap = max(reported) - min(reported)
        top = max(max(reported), base, paper) + (gap*0.2)
        bottom = min(min(reported), base, paper) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
        plt.axhline(y=paper, linestyle='--', c='red')
    
    plt.title(f'{exp}-{task} ({round(base,4)})')
    plt.savefig(f'images/{exp}/{task}', format='png', bbox_inches='tight')
    plt.show()

In [11]:
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        reported = task_results[metric]
        base = task_results[f'base-{metric}']
        print_bold(task, metric, ': b -', round(base * 100, 2), 'h -',round(task_results[metric][0] * 100, 2), 'n -', round(task_results[metric][-1] * 100, 2))


CoLA mcc : b - 66.04 h - 61.57 n - 0


SST-2 acc : b - 94.95 h - 95.76 n - 79.33


MRPC f1 : b - 93.12 h - 92.28 n - 81.22


STS-B spearmanr : b - 92.01 h - 90.91 n - 11.19


RTE acc : b - 86.28 h - 83.03 n - 48.3



In [12]:
import copy 

layer_90 = []
layer_95 = []

threshold_90 = 0.9
threshold_95 = 0.95
x_axis.reverse()

for task in tasks:
#     print_bold(task)
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        flag_90 = True
        flag_95 = True

        for ind, val in enumerate(reported):

            if val/base > threshold_90 and flag_90:
                flag_90 = False
                layer_90.append(ind)
                results[task]['90%'] = ind

            if val/base > threshold_95 and flag_95:
                flag_95 = False
                layer_95.append(ind)
                results[task]['95%'] = ind

        if flag_90:
            print(task, "Fails to achieve 90% threshold", reported[-1]/base)
            layer_90.append(len(reported)-1)
            results[task]['90%'] = "-"

        if flag_95:
            print(task, "Fails to achieve 95% threshold", reported[-1]/base)
            layer_95.append(len(reported)-1)
            results[task]['95%'] = "-"


            
print(x_axis)
            
            
print(layer_90)
min_layer_ind_90 = max(layer_90)
print("layer_90 ", min_layer_ind_90, 'layer:', x_axis[min_layer_ind_90], round((1-(min_layer_ind_90/num_layers)) * 100, 2), '%')

print(layer_95)
min_layer_ind_95 = max(layer_95)
print("layer_95 ", min_layer_ind_95, 'layer:', x_axis[min_layer_ind_95], round((1-(min_layer_ind_95/num_layers)) * 100, 2), '%')


firsts = []
seconds = []
    
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        if task != "CoLA":
            first = round(100*reported[0]/base, 2)
            second = round(100*reported[1]/base, 2)
            firsts.append(first)
            seconds.append(second)
            
        print_bold(task, base)
        print('\t90', reported[min_layer_ind_90], round(reported[min_layer_ind_90]/base * 100, 2))
        print('\t95', reported[min_layer_ind_95], round(reported[min_layer_ind_95]/base * 100, 2))
        
print_bold(len(firsts), np.mean(firsts), np.mean(seconds), round(np.mean(seconds) - np.mean(firsts),2))

CoLA Fails to achieve 95% threshold 0.9323942429359162
['none', '23', '22', '21', '20', '19', '18', '17', '16', '15', '14', '13', '12']
[5, 1, 1, 2, 9]
layer_90  9 layer: 15 62.5 %
[12, 1, 5, 5, 12]
layer_95  12 layer: 12 50.0 %

CoLA 0.6603613028668636

	90 0.6081727293932068 92.1
	95 0.6157170770507245 93.24

SST-2 0.9495412844036697

	90 0.948394495412844 99.88
	95 0.9575688073394495 100.85

MRPC 0.9312169312169313

	90 0.9113043478260869 97.86
	95 0.9228070175438596 99.1

STS-B 0.9200804739971572

	90 0.9067461676538378 98.55
	95 0.9090678116027199 98.8

RTE 0.8628158844765343

	90 0.8014440433212996 92.89
	95 0.8303249097472925 96.23

4 59.724999999999994 86.5575 26.83



In [13]:
for task in ["STS-B"]:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        print(task_results[metric][-1])
        print(task_results[metric][-2])

0.11190994192398618
0.824179645266765


In [15]:
print("\\begin{table*}\n\\centering\n\\setlength{\\tabcolsep}{2pt}\n\\scalebox{0.88}{\n\\begin{tabular}{lc|ccccccccccccc|cc}\n\\toprule[1pt]\ntask & base & 12-23 & 13-23 & 14-23 & 15-23 & 16-23 & 17-23 & 18-23 & 19-23 & 20-23 & 21-23 & 22-23 & 23-23 & none & 90\\% & 95\\% \\\\\n\\midrule")

avg_performance = []

for task in tasks:
    m = metrics[task][0]
    base_key = f"base-{m}"
    row = f"{task} & "
    
    row += "{:0.2f}".format(round(results[task][base_key] * 100, 2))
    
    for ind, val in enumerate(results[task][m]):
        row += " & {:0.2f}".format(round(val * 100,2))
        
        if len(avg_performance) == ind:
            avg_performance.append([])
            
            
        percent = (val / results[task][base_key]) * 100
        avg_performance[ind].append(percent)
        
    row += "& {}".format(results[task]["90%"])
    row += "& {}".format(results[task]["95%"])
        
    row += " \\\\"
    print(row)

print("\\midrule")

row = "&"

for perf in avg_performance:
    row += " & {:0.2f}".format(round(np.mean(perf) ,2))
    
row += " \\\\"

print(row)
    
    
print("\\end{tabular}\n}\n\\caption{MTDNN-roBERTa-large on GLUE}\n\\label{table:finetune-all}\n\\end{table*}")

\begin{table*}
\centering
\setlength{\tabcolsep}{2pt}
\scalebox{0.88}{
\begin{tabular}{lc|ccccccccccccc|cc}
\toprule[1pt]
task & base & 12-23 & 13-23 & 14-23 & 15-23 & 16-23 & 17-23 & 18-23 & 19-23 & 20-23 & 21-23 & 22-23 & 23-23 & none & 90\% & 95\% \\
\midrule
CoLA & 66.04 & 61.57 & 59.82 & 57.79 & 60.82 & 61.07 & 61.82 & 60.58 & 60.32 & 58.87 & 58.67 & 54.39 & 48.80 & 0.00& 5& - \\
SST-2 & 94.95 & 95.76 & 95.64 & 94.72 & 94.84 & 95.07 & 95.07 & 94.50 & 94.64 & 93.84 & 93.55 & 94.04 & 92.32 & 79.33& 1& 1 \\
MRPC & 93.12 & 92.28 & 92.10 & 91.48 & 91.13 & 90.78 & 91.38 & 90.97 & 89.94 & 86.39 & 84.95 & 85.19 & 84.17 & 81.22& 1& 5 \\
STS-B & 92.01 & 90.91 & 90.71 & 90.64 & 90.67 & 90.00 & 89.58 & 88.53 & 87.89 & 85.78 & 84.32 & 83.61 & 82.42 & 11.19& 2& 5 \\
RTE & 86.28 & 83.03 & 81.59 & 79.42 & 80.14 & 77.26 & 77.26 & 74.01 & 69.10 & 66.57 & 64.48 & 61.59 & 59.57 & 48.30& 9& 12 \\
\midrule
& & 97.64 & 96.67 & 95.21 & 96.26 & 95.49 & 95.75 & 94.19 & 92.64 & 90.23 & 88.99 & 87.03 & 84.03